# Library

In [119]:
import pandas as pd
import numpy as np
import os
import re
import collections
#from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
import itertools 
from nltk.tokenize import word_tokenize
from nltk.tag.stanford import StanfordPOSTagger
from string import punctuation
from textblob import TextBlob

from nltk import ngrams

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Functions

In [120]:
#java path for standford tagger
java_path = "C:\Program Files (x86)\Java\jre1.8.0_201"
os.environ['JAVAHOME'] = java_path

In [118]:
# Load Standford Spanish POS tagger
_model_filename = r'C:/Users/cx02274/Documents/master/stanford-postagger/models/spanish-ud.tagger'
_path_to_jar = r'C:/Users/cx02274/Documents/master/stanford-postagger/stanford-postagger-4.1.0.jar'
st = StanfordPOSTagger(model_filename=_model_filename, path_to_jar=_path_to_jar,encoding='utf8')

In [12]:
def read_texts(path):
    data = []
    file_name = os.listdir(path)

    for name in file_name:
        if name.endswith('.txt'):
            with open(path + name,encoding="utf8") as f:
                text = f.read()
                data.append({'nombre':name.replace('.txt',''), 'texto':text})

    df = pd.DataFrame(data)
    return df

# Load Data

### Trainning

318 clinical cases

In [3]:
train_abbr = pd.read_csv("../datasets/trainning_set/clinical_cases.abbreviations.training_set.tsv", sep = '\t')
train_met = pd.read_csv("../datasets/trainning_set/clinical_cases.metadata.training_set.tsv", sep = '\t')
train_rel = pd.read_csv("../datasets/trainning_set/clinical_cases.relations.training_set.tsv", sep = '\t')

In [4]:
train_met = train_met.rename(columns = {'# Document_ID': 'doc_id'})

In [5]:
train_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1139-76322015000500009-1.txt,1.txt,1139-7632,2015-12-01,Pediatría Atención Primaria v.17 n.68 2015,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-05582008000400007-2.txt,2.txt,1130-0558,2008-08-01,Revista Española de Cirugía Oral y Maxilofacia...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S0210-48062006000100012-1.txt,1.txt,0210-4806,2006-01-01,Actas Urológicas Españolas v.30 n.1 2006,http://scielo.isciii.es/scielo.php?script=sci_...
3,S0213-12852003000500002-1.txt,1.txt,0213-1285,2003-10-01,Avances en Odontoestomatología v.19 n.5 2003,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0212-71992005000400007-1.txt,1.txt,0212-7199,2005-04-01,Anales de Medicina Interna v.22 n.4 2005,http://scielo.isciii.es/scielo.php?script=sci_...


In [6]:
train_rel = train_rel.reset_index()

In [7]:
train_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [8]:
train_rel = train_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [9]:
train_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S1130-01082009000400014-1,SHORT_FORM,476,NPT,SHORT-LONG,LONG_FORM,454.0,474.0,nutrición parenteral
1,S1130-63432016000100009-1,SHORT_FORM,614,NIHSS,SHORT-LONG,LONG_FORM,621.0,662.0,National Institute of Health Stroke Scale
2,S1139-76322017000200007-1,SHORT_FORM,1145,CMV,SHORT-LONG,LONG_FORM,1128.0,1143.0,citomegalovirus
3,S1139-76322017000200007-1,SHORT_FORM,1243,VSG,SHORT-LONG,LONG_FORM,1206.0,1241.0,velocidad de sedimentación globular
4,S1139-76322017000200007-1,SHORT_FORM,1300,IGRA,SHORT-LONG,LONG_FORM,1267.0,1298.0,interferon-gamma release assays


In [10]:
train_abbr = train_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [18]:
train_raw = read_texts("../datasets/trainning_set/training_set.raw_text/txt/")

In [19]:
train_raw = train_raw.rename(columns = {'nombre': 'doc_id'})

In [20]:
train_raw.head()

,doc_id,texto
0,S0004-06142005000900013-1,Se trata de una mujer de 29 años sometida a un...
1,S0004-06142005000900015-1,"Varón de 36 años, sin antecedentes de interés,..."
2,S0004-06142005000900016-1,Mujer de 29 años con antecedentes de ulcus duo...
3,S0004-06142005001000011-1,Varón de 58 años de edad en el momento del tra...
4,S0004-06142005001000011-3,Mujer de 42 años en el momento de someterse a ...


# Sub-track 1

## Found abbreviations (Short Forms)

#### Text preprocessing

In [81]:
#tokenize text
train_raw['tokens'] = train_raw['texto'].map(lambda x: word_tokenize(x))

In [82]:
#Remove stop words
#nltk.download('stopwords')
swords = list(set(stopwords.words('spanish')))
train_raw['texto_clean'] = train_raw['tokens'].map(lambda x: ' '.join([w for w in x if w not in swords]))

In [83]:
#Remove white spaces
train_raw['texto_clean'] = train_raw['texto_clean'].map(lambda x: x.strip())

In [84]:
#Split text in sentences
train_raw['texto_sentences'] = train_raw['texto_clean'].map(lambda x: re.split(r"\.|\?|\!", x))

In [165]:
#POS tagging
train_raw['texto_POS'] = train_raw['texto_clean'].map(lambda x: st.tag(nltk.word_tokenize(x)))

In [166]:
train_raw.head()

,doc_id,texto,tokens,texto_clean,texto_sentences,texto_POS
0,S0004-06142005000900013-1,Se trata de una mujer de 29 años sometida a un...,"[Se, trata, de, una, mujer, de, 29, años, some...",Se trata mujer 29 años sometida estudio ecográ...,[Se trata mujer 29 años sometida estudio ecogr...,"[(Se, PRON), (trata, VERB), (mujer, NOUN), (29..."
1,S0004-06142005000900015-1,"Varón de 36 años, sin antecedentes de interés,...","[Varón, de, 36, años, ,, sin, antecedentes, de...","Varón 36 años , antecedentes interés , estudia...","[Varón 36 años , antecedentes interés , estudi...","[(Varón, PROPN), (36, NUM), (años, NOUN), (,, ..."
2,S0004-06142005000900016-1,Mujer de 29 años con antecedentes de ulcus duo...,"[Mujer, de, 29, años, con, antecedentes, de, u...",Mujer 29 años antecedentes ulcus duodenal estr...,[Mujer 29 años antecedentes ulcus duodenal est...,"[(Mujer, NOUN), (29, NUM), (años, NOUN), (ante..."
3,S0004-06142005001000011-1,Varón de 58 años de edad en el momento del tra...,"[Varón, de, 58, años, de, edad, en, el, moment...","Varón 58 años edad momento trasplante , 5 octu...","[Varón 58 años edad momento trasplante , 5 oct...","[(Varón, PROPN), (58, NUM), (años, NOUN), (eda..."
4,S0004-06142005001000011-3,Mujer de 42 años en el momento de someterse a ...,"[Mujer, de, 42, años, en, el, momento, de, som...",Mujer 42 años momento someterse trasplante hep...,[Mujer 42 años momento someterse trasplante he...,"[(Mujer, NOUN), (42, NUM), (años, NOUN), (mome..."


#### POS

In [123]:
prueba = train_raw.iloc[50]['texto_clean']
prueba

'Se trata mujer 66 años diabética antecedentes hepatitis virus C etiología postransfusional . En Enero 2006 presentaba síndrome miccional pertinaz respondía tratamiento realizó cistoscopia posterior RTU . Fue diagnosticada carcinoma vesical pTa G3 tras inició tratamiento BCG intravesical . La RTU control tras finalizar tratamiento BCG detectó presencia múltiples focos carcinoma vejiga confirmándose diagnóstico carcinoma urotelial invadía capa muscular ( pT2 G3 ) . En Julio 2006 sometida cistectomía radical derivación urinaria mediante ureterosigmoidostomía tipo Mainz II . La anatomía patológica demostró carcinoma urotelial alto grado invasión capa muscular afectación ganglios linfáticos resecados , trataba pT2a N1 . No detectaron metástasis distancia . Dos semanas después intervención , acudió desorientación tendencia sueño contexto cuadro compatible infección vías urinarias . Tras 24 horas observación , hidratación intravenosa antibioterapia recuperó pudo ser dada alta antibiótico ora

In [124]:
text = st.tag(nltk.word_tokenize(prueba))
print(text)

[('Se', 'PRON'), ('trata', 'VERB'), ('mujer', 'NOUN'), ('66', 'NUM'), ('años', 'NOUN'), ('diabética', 'VERB'), ('antecedentes', 'NOUN'), ('hepatitis', 'NOUN'), ('virus', 'NOUN'), ('C', 'PROPN'), ('etiología', 'VERB'), ('postransfusional', 'ADJ'), ('.', 'PUNCT'), ('En', 'ADP'), ('Enero', 'PROPN'), ('2006', 'NUM'), ('presentaba', 'VERB'), ('síndrome', 'NOUN'), ('miccional', 'ADJ'), ('pertinaz', 'ADJ'), ('respondía', 'VERB'), ('tratamiento', 'NOUN'), ('realizó', 'VERB'), ('cistoscopia', 'NOUN'), ('posterior', 'ADJ'), ('RTU', 'PROPN'), ('.', 'PUNCT'), ('Fue', 'AUX'), ('diagnosticada', 'VERB'), ('carcinoma', 'NOUN'), ('vesical', 'ADJ'), ('pTa', 'PROPN'), ('G3', 'PROPN'), ('tras', 'ADP'), ('inició', 'VERB'), ('tratamiento', 'NOUN'), ('BCG', 'PROPN'), ('intravesical', 'ADJ'), ('.', 'PUNCT'), ('La', 'DET'), ('RTU', 'PROPN'), ('control', 'NOUN'), ('tras', 'ADP'), ('finalizar', 'VERB'), ('tratamiento', 'NOUN'), ('BCG', 'PROPN'), ('detectó', 'VERB'), ('presencia', 'NOUN'), ('múltiples', 'ADJ'), (

### Detect acronym with patterns and heuristic search

In [160]:
def acron_detect(concept):
    if ((len(concept) >= 2 and len(concept) <= 10) and sum(x.isupper() for x in concept) >= 1 and any(x.isalpha() for x in concept) and sum(x.isdigit() for x in concept) <= 1) and \
    (sum(c.isspace() for c in concept) <= 1):
        print(concept, "is an acronym")
#     else:
#         print(concept, "is NOT and acronym")

In [167]:
tokens = train_raw.iloc[0]['texto_POS']

In [161]:
for x in tokens:
    acron_detect(x)

Se is an acronym
Durante is an acronym
La is an acronym
Se is an acronym
En is an acronym
Las is an acronym
Se is an acronym
Con is an acronym
Los is an acronym
El is an acronym
No is an acronym
El is an acronym
DAKO is an acronym


In [171]:
rules = [lambda concept: sum(x.isdigit() for x in concept) <= 1, # could have one digit
        lambda concept: any(x.isalpha() for x in concept),  # must have at least one letter
        lambda concept: sum(x.isupper() for x in concept) >= 1,  # must have at least one uppercase
        lambda concept: sum(x.isspace() for x in concept) <= 1,  # could have just one white space
        lambda concept: (len(concept) >= 2 and len(concept) <= 10), # must be more than 2 characters and at least 10 characters
         
        ]

In [164]:
# [p for p in [r('DAKO') for r in rules]]

In [174]:
for token in tokens:
    if all(rule(token[0]) for rule in rules) and token[1] == 'PROPN':
        print(token)

('DAKO', 'PROPN')


### Regex

In [ ]:
#patron = r'\(([A-Z]{2,8})\)'
#patron1 = r'\s[A-Z]{1,3}\s'
#patron2 = r'\s[a-z]{1,3}\s'
patron3 = r'[A-Z]{2,8}'
patron4 = r'\s[a-z]{1,2}\s'
patron5 = r'[a-z]+\-[A-Z]{1,8}'
patron6 = r'[a-z]+\/[a-z]+'
patron7 = r'[A-Z]?[a-z]{1,4}[A-Z]+[a-z]*[1-9]*'
patron8 = r'\/[a-z]*[A-Z]*'


# create a list with them
regexes = [ patron3, patron4, patron5, patron6, patron7,patron8]
for i in regexes:
    generic_re = re.compile("%s|%s|%s|%s|%s|%s" % (patron3, patron4, patron5, patron6, patron7,patron8))

**Get Short Formns with a regex in each text**

In [ ]:
train_raw['abrev'] = train_raw['texto'].map(lambda x: generic_re.findall(x))